In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot
%matplotlib inline 

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

In [3]:
cancer=load_breast_cancer()



In [4]:
df = pd.DataFrame(cancer['data'],columns=cancer['feature_names'])
label=cancer["target"]

In [5]:
print(df.head())

   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst radius  worst texture  worst perimeter  \
0           

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    label, test_size=0.30, 
                                                    random_state=101)



In [7]:
from sklearn.tree import DecisionTreeClassifier

Dtree = DecisionTreeClassifier()


In [8]:
Dtree.fit(X_train,y_train)

DecisionTreeClassifier()

In [9]:
predictions = Dtree.predict(X_test)
print("Accuracy = "+ str(accuracy_score(y_test,predictions)))

Accuracy = 0.9064327485380117


In [10]:
#let’s include a genetic algorithm in the process:

In [11]:
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)
        chromosome[:int(0.3*n_feat)]=False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

def fitness_score(population):
    scores = []
    for chromosome in population:
        Dtree.fit(X_train.iloc[:,chromosome],y_train)
        predictions = Dtree.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds,:][::-1])

def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen

def crossover(pop_after_sel):
    population_nextgen=pop_after_sel
    for i in range(len(pop_after_sel)):
        child=pop_after_sel[i]
        child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]
        population_nextgen.append(child)
    return population_nextgen

def mutation(pop_after_cross,mutation_rate):
    population_nextgen = []
    for i in range(0,len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j]= not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen

def generations(size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, y_train, y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print(scores[:2])
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

In [12]:
chromo,score=generations(size=200,n_feat=30,n_parents=100,mutation_rate=0.10,
                     n_gen=38,X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test)

[0.9532163742690059, 0.9532163742690059]
[0.9590643274853801, 0.9590643274853801]
[0.9532163742690059, 0.9532163742690059]
[0.9649122807017544, 0.9590643274853801]
[0.9590643274853801, 0.9590643274853801]
[0.9532163742690059, 0.9532163742690059]
[0.9590643274853801, 0.9532163742690059]
[0.9590643274853801, 0.9590643274853801]
[0.9532163742690059, 0.9532163742690059]
[0.9590643274853801, 0.9532163742690059]
[0.9590643274853801, 0.9532163742690059]
[0.9590643274853801, 0.9590643274853801]
[0.9590643274853801, 0.9532163742690059]
[0.9532163742690059, 0.9473684210526315]
[0.9532163742690059, 0.9532163742690059]
[0.9590643274853801, 0.9532163742690059]
[0.9590643274853801, 0.9532163742690059]
[0.9590643274853801, 0.9590643274853801]
[0.9590643274853801, 0.9590643274853801]
[0.9590643274853801, 0.9590643274853801]
[0.9649122807017544, 0.9590643274853801]
[0.9590643274853801, 0.9532163742690059]
[0.9590643274853801, 0.9590643274853801]
[0.9532163742690059, 0.9532163742690059]
[0.9532163742690

In [13]:
Dtree.fit(X_train.iloc[:,chromo[-1]],y_train)
predictions = Dtree.predict(X_test.iloc[:,chromo[-1]])
print("Accuracy score after genetic algorithm is= "+str(accuracy_score(y_test,predictions)))

Accuracy score after genetic algorithm is= 0.9415204678362573
